In [1]:
import pandas as pd
import numpy as np

/var/folders/zp/c97k147160ddnklbfc2rk3hr0000gn/T/ipykernel_7229/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_pickle('data/SBAmodel.pickle')
pd.set_option('display.max_columns', None)
df.head(1)

,Name,City,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv
0,ABC HOBBYCRAFT,EVANSVILLE,47711,FIFTH THIRD BANK,38,45,1997-02-28,1997,84,4,0,0,0,1,0,0,0,1,60000.0


# Split

In [42]:
from sklearn.model_selection import train_test_split
from feature_engine.selection import DropFeatures


X = df.drop(columns=['MIS_Status'])
y = df.MIS_Status

#X = DropFeatures(features_to_drop=['Name', 'City', 'Bank']).fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=69
)

X_train.head(1)

,Name,City,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv
217398,"TANEYCOMO PIZZA,INC.",HOLLISTER,65672,LIBERTY BANK,27,72,2006-11-20,2007,110,1,0,0,0,64900,2,0,0,662000.0


In [43]:
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FunctionTransformer, make_pipeline

set_config(transform_output = "pandas")

drop = DropFeatures(features_to_drop=['Name', 'City', 'Bank'])
transformers = ColumnTransformer(
    transformers = [
        ('date', FunctionTransformer(lambda x: x.astype(int)), ['ApprovalDate'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False)

scaler = StandardScaler()
estimator = RandomForestClassifier(max_depth=5)

pipe = make_pipeline(
            drop,
            transformers,
            scaler,
            estimator)

pipe.fit(X_train, y_train)

Pipeline(steps=[('dropfeatures',
                 DropFeatures(features_to_drop=['Name', 'City', 'Bank'])),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('date',
                                                  FunctionTransformer(func=<function <lambda> at 0x2a0361b20>),
                                                  ['ApprovalDate'])],
                                   verbose_feature_names_out=False)),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=5))])

In [44]:
from sklearn.metrics import classification_report

y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.42      0.56     31512
           1       0.89      0.98      0.93    148321

    accuracy                           0.88    179833
   macro avg       0.86      0.70      0.74    179833
weighted avg       0.88      0.88      0.87    179833



In [45]:
X_train_transf = pipe[:-1].fit_transform(X_train, y_train)
feat_labels = X_train_transf.columns

f = pd.DataFrame({'features': np.array(list(feat_labels)), 'score': pipe.named_steps['randomforestclassifier'].feature_importances_})
f.sort_values(by=['score'], ascending=False, inplace=True)
f

ValueError: All arrays must be of the same length